# Modified from:
https://github.com/huggingface/notebooks/blob/main/examples/token_classification.ipynb

If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers and 🤗 Datasets. Uncomment the following cell and run it.

In [ ]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.1 MB/s eta 0:00:00


In [ ]:
! pip install datasets transformers seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 107.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 74.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 101.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8

If you're opening this notebook locally, make sure your environment has an install from the last version of those libraries.

To be able to share your model with the community and generate results like the one shown in the picture below via the inference API, there are a few more steps to follow.

First you have to store your authentication token from the Hugging Face website (sign up [here](https://huggingface.co/join) if you haven't already!) then execute the following cell and input your username and password:

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


Then you need to install Git-LFS. Uncomment the following instructions:

In [ ]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.9.2-1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


Make sure your version of Transformers is at least 4.11.0 since the functionality was introduced in that version:

In [ ]:
import transformers

print(transformers.__version__)

4.28.1


You can find a script version of this notebook to fine-tune your model in a distributed fashion using multiple GPUs or TPUs [here](https://github.com/huggingface/transformers/tree/master/examples/token-classification).

We also quickly upload some telemetry - this tells us which examples and software versions are getting used so we know where to prioritize our maintenance efforts. We don't collect (or care about) any personally identifiable information, but if you'd prefer not to be counted, feel free to skip this step or delete this cell entirely.

In [ ]:
from transformers.utils import send_example_telemetry

send_example_telemetry("token_classification_notebook", framework="pytorch")

# Fine-tuning a model on a token classification task

In this notebook, we will see how to fine-tune one of the [🤗 Transformers](https://github.com/huggingface/transformers) model to a token classification task, which is the task of predicting a label for each token.

![Widget inference representing the NER task](images/token_classification.png)

The most common token classification tasks are:

- NER (Named-entity recognition) Classify the entities in the text (person, organization, location...).
- POS (Part-of-speech tagging) Grammatically classify the tokens (noun, verb, adjective...)
- Chunk (Chunking) Grammatically classify the tokens and group them into "chunks" that go together

We will see how to easily load a dataset for these kinds of tasks and use the `Trainer` API to fine-tune a model on it.

This notebook is built to run on any token classification task, with any model checkpoint from the [Model Hub](https://huggingface.co/models) as long as that model has a version with a token classification head and a fast tokenizer (check on [this table](https://huggingface.co/transformers/index.html#bigtable) if this is the case). It might just need some small adjustments if you decide to use a different dataset than the one used here. Depending on you model and the GPU you are using, you might need to adjust the batch size to avoid out-of-memory errors. Set those three parameters, then the rest of the notebook should run smoothly:

In [ ]:
!wget https://dl.fbaipublicfiles.com/biolm/RoBERTa-large-PM-M3-Voc-hf.tar.gz

--2023-04-23 13:21:55--  https://dl.fbaipublicfiles.com/biolm/RoBERTa-large-PM-M3-Voc-hf.tar.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.227.254.68, 13.227.254.76, 13.227.254.55, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.227.254.68|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 839592355 (801M) [application/gzip]
Saving to: ‘RoBERTa-large-PM-M3-Voc-hf.tar.gz’

RoBERTa-large-PM-M3 100%[===================>] 800.70M   201MB/s    in 4.1s    

2023-04-23 13:21:59 (197 MB/s) - ‘RoBERTa-large-PM-M3-Voc-hf.tar.gz’ saved [839592355/839592355]



In [ ]:
!tar -xf RoBERTa-large-PM-M3-Voc-hf.tar.gz

In [ ]:
task = "ner" # Should be one of "ner", "pos" or "chunk"
model_checkpoint = "/content/RoBERTa-large-PM-M3-Voc/RoBERTa-large-PM-M3-Voc-hf"
batch_size = 16

## Loading the dataset

We will use the [🤗 Datasets](https://github.com/huggingface/datasets) library to download the data and get the metric we need to use for evaluation (to compare our model to the benchmark). This can be easily done with the functions `load_dataset` and `load_metric`.  

In [ ]:
# from datasets import load_dataset, load_metric, DatasetDict, Dataset
# orig_dataset = load_dataset("Temporarily/IR-NER-Peptents")

In [ ]:
# import json
# import re
# keep_list = ['Diagnostic_procedure', 'Medication', 'Lab_value', 'Detailed_description']
# with open('data.jsonl', 'w') as outfile:
#     for i, (tokens, label) in enumerate(zip(orig_dataset['train']['tokens'], orig_dataset['train']['tags'])):
#         entry = {'tokens':tokens, 'tags':[]}
#         entry['tags'] = [re.sub('Diagnostic_procedure', 'Detailed_description', l) if (l[2:] in keep_list) else 'O' for l in label]
#         json.dump(entry, outfile)
#         outfile.write('\n')

For our example here, we'll use the [CONLL 2003 dataset](https://www.aclweb.org/anthology/W03-0419.pdf). The notebook should work with any token classification dataset provided by the 🤗 Datasets library. If you're using your own dataset defined from a JSON or csv file (see the [Datasets documentation](https://huggingface.co/docs/datasets/loading_datasets.html#from-local-files) on how to load them), it might need some adjustments in the names of the columns used.

In [ ]:
from datasets import load_dataset, load_metric, DatasetDict, Dataset, concatenate_datasets

# train_datasets_maccrobat = load_dataset("ktgiahieu/IR_dataset_v2" , split='train[:400]')
# train_datasets = load_dataset("ktgiahieu/IR_dataset_v2" , split='train[400:-40]')
# test_datasets = load_dataset("ktgiahieu/IR_dataset_v2" , split='train[-40:]')

# datasets = DatasetDict({"train":concatenate_datasets([train_datasets_maccrobat]+ [train_datasets]*3),"test":test_datasets})


datasets = load_dataset("ktgiahieu/IR_combined_filtered")
datasets = datasets['train'].train_test_split(test_size=0.1)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/ktgiahieu___json/ktgiahieu--IR_combined_filtered-0b186493af786fd6/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

The `datasets` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set.

In [ ]:
datasets

DatasetDict({
    train: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 162
    })
    test: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 18
    })
})

We can see the training, validation and test sets all have a column for the tokens (the input texts split into words) and one column of labels for each kind of task we introduced before.

To access an actual element, you need to select a split first, then give an index:

In [ ]:
from datasets import Dataset, ClassLabel, Value, DatasetDict

# Define the ClassLabel feature with the label names
label_list = ['B-Term','B-Lab_value','B-Chemical','I-Term','I-Lab_value','I-Chemical','O']
label = ClassLabel(names=label_list)
# Define a new feature with the numeric labels
numeric_labels_feature = Value("int32")

In [ ]:
datasets = datasets.map(lambda example: {"tokens": example['tokens'],
    "tags": example["tags"],
    "numeric_tags": [label.encode_example(x) for x in example["tags"]],
})

Map:   0%|          | 0/162 [00:00<?, ? examples/s]

Map:   0%|          | 0/18 [00:00<?, ? examples/s]

In [ ]:
datasets

DatasetDict({
    train: Dataset({
        features: ['tokens', 'tags', 'numeric_tags'],
        num_rows: 162
    })
    test: Dataset({
        features: ['tokens', 'tags', 'numeric_tags'],
        num_rows: 18
    })
})

The labels are already coded as integer ids to be easily usable by our model, but the correspondence with the actual categories is stored in the `features` of the dataset:

Since the labels are lists of `ClassLabel`, the actual names of the labels are nested in the `feature` attribute of the object above:

In [ ]:
# label_list = datasets["train"].features[f"{task}_tags"].feature.names
# label_list

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset (automatically decoding the labels in passing).

In [ ]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(datasets["train"])

,tokens,tags,numeric_tags
0,"[In, specific, aspects, ,, the, kifunensine, is, present, in, the, culture, medium, at, a, concentration, of, about, 0.1, µg, /, ml, to, about, 10, µg, /, ml, ., In, some, aspects, ,, the, kifunensine, is, present, in, the, culture, medium, at, a, concentration, of, about, 0.25, µg, /, ml, to, about, 2, µg, /, ml, ., In, some, aspects, ,, the, kifunensine, is, present, in, the, culture, medium, at, a, concentration, of, about, 0.01, µg, /, ml, to, about, 1, mg, /, ml, .]","[O, O, O, O, O, B-Term, O, O, O, O, O, O, O, O, O, O, O, B-Lab_value, I-Lab_value, I-Lab_value, I-Lab_value, O, O, B-Lab_value, I-Lab_value, I-Lab_value, I-Lab_value, O, O, O, O, O, O, B-Term, O, O, O, O, O, O, O, O, O, O, O, B-Lab_value, I-Lab_value, I-Lab_value, I-Lab_value, O, O, B-Lab_value, I-Lab_value, I-Lab_value, I-Lab_value, O, O, O, O, O, O, B-Term, O, O, O, O, O, O, O, O, O, O, O, B-Lab_value, I-Lab_value, I-Lab_value, I-Lab_value, O, O, B-Lab_value, I-Lab_value, I-Lab_value, I-Lab_value, O]","[6, 6, 6, 6, 6, 0, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 1, 4, 4, 4, 6, 6, 1, 4, 4, 4, 6, 6, 6, 6, 6, 6, 0, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 1, 4, 4, 4, 6, 6, 1, 4, 4, 4, 6, 6, 6, 6, 6, 6, 0, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 1, 4, 4, 4, 6, 6, 1, 4, 4, 4, 6]"
1,"[In, specific, aspects, ,, the, lentiviral, vector, genome, is, derived, from, HIV-1, .]","[O, O, O, O, O, B-Term, I-Term, I-Term, O, O, O, B-Term, O]","[6, 6, 6, 6, 6, 0, 3, 3, 6, 6, 6, 0, 6]"
2,"[In, the, present, application, ,, the, polymyxin, B, fermentation, liquid, used, is, prepared, by, pure, strain, fermentation, for, 40, -, 50, hours, using, bacillus, polymyxa, with, flour, and, cottonseed, meal, as, main, raw, materials, to, obtain, the, polymyxin, B, fermentation, liquid, ., The, control, sample, is, a, polymyxin, B, sulfate, raw, material, sample, and, a, commercially, available, sample, .]","[O, O, O, O, O, O, B-Chemical, I-Chemical, B-Term, O, O, O, O, O, B-Term, I-Term, I-Term, O, O, O, O, O, O, B-Term, I-Term, O, O, O, O, O, O, O, O, O, O, O, O, B-Chemical, I-Chemical, B-Term, I-Term, O, O, O, O, O, O, B-Chemical, I-Chemical, I-Chemical, O, O, O, O, O, O, O, O, O]","[6, 6, 6, 6, 6, 6, 2, 5, 0, 6, 6, 6, 6, 6, 0, 3, 3, 6, 6, 6, 6, 6, 6, 0, 3, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 2, 5, 0, 3, 6, 6, 6, 6, 6, 6, 2, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6]"
3,"[In, some, or, any, of, the, embodiments, described, herein, ,, the, pseudotyped, lentiviral, vector, particles, also, more, efficiently, transduce, cells, expressing, mouse, SIGNR1, compared, to, cells, not, expressing, mouse, SIGNR1, .]","[O, O, O, O, O, O, O, O, O, O, O, B-Term, I-Term, I-Term, I-Term, O, O, O, O, O, O, O, B-Term, O, O, O, O, O, O, B-Term, O]","[6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 0, 3, 3, 3, 6, 6, 6, 6, 6, 6, 6, 0, 6, 6, 6, 6, 6, 6, 0, 6]"
4,"[Methods, of, generating, pseudotyped, lentiviral, vector, particles]","[O, O, O, B-Term, I-Term, I-Term, I-Term]","[6, 6, 6, 0, 3, 3, 3]"
5,"[In, some, aspects, ,, pseudotyped, lentiviral, vector, particle, further, comprises, a, Rev, protein, .]","[O, O, O, O, B-Term, I-Term, I-Term, I-Term, O, O, O, B-Term, I-Term, O]","[6, 6, 6, 6, 0, 3, 3, 3, 6, 6, 6, 0, 3, 6]"
6,"[1, ., An, anhydrous, crystal, 1, of, polymyxin, B1, sulfate, ,, , characterized, in, that, ,, said, anhydrous, crystal, 1, has, an, X, -, ray, powder, diffraction, pattern, having, diffraction, peaks, at, 3.396, ,, 4.895, and, 6.903, expressed, by, 2θ, degree, using, Cu, -, Ka, radiation, ;, and, preferably, ,, said, crystal, 1, has, an, X, -, ray, powder, diffraction, pattern, expressed, by, 2θ, degree, using, Cu, -, Ka, radiation, as, shown, in, Figure, 2A, ;, preferably, ,, said, anhydrous, crystal, 1, has, an, infrared, absorption, spectrum, having, characteristic, bands, at, 1071.93, cm, -1, ,, 1242.91, cm, -1, ,, 1384.25, ...]","[O, O, O, O, O, O, O, B-Chemical, I-Chemical, I-Chemical, O, O, O, O, O, O, O, O, O, O, O, O, B-Term, I-Term, I-Term, I-Term, I-Term, I-Term, O, O, O, O, B-Lab_value,

## Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [ ]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


The following assertion ensures that our tokenizer is a fast tokenizers (backed by Rust) from the 🤗 Tokenizers library. Those fast tokenizers are available for almost all models, and we will need some of the special features they have for our preprocessing.

In [ ]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

You can check which type of models have a fast tokenizer available and which don't on the [big table of models](https://huggingface.co/transformers/index.html#bigtable).

You can directly call this tokenizer on one sentence:

In [ ]:
tokenizer("Hello, this is one sentence!")

{'input_ids': [0, 427, 24017, 15, 491, 345, 818, 17812, 4, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Depending on the model you selected, you will see different keys in the dictionary returned by the cell above. They don't matter much for what we're doing here (just know they are required by the model we will instantiate later), you can learn more about them in [this tutorial](https://huggingface.co/transformers/preprocessing.html) if you're interested.

If, as is the case here, your inputs have already been split into words, you should pass the list of words to your tokenzier with the argument `is_split_into_words=True`:

In [ ]:
tokenizer(["Hello", ",", "this", "is", "one", "sentence", "split", "into", "words", "."], is_split_into_words=True)

{'input_ids': [0, 427, 24017, 3267, 491, 345, 818, 17812, 9490, 860, 6786, 2245, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Note that transformers are often pretrained with subword tokenizers, meaning that even if your inputs have been split into words already, each of those words could be split again by the tokenizer. Let's look at an example of that:

In [ ]:
example = datasets["train"][4]
print(example["tokens"])

['Preferably', ',', 'in', 'step', '2', ')', 'of', 'the', 'method', 'of', 'the', 'present', 'invention', ',', 'the', 'pH', 'of', 'the', 'desorption', 'liquid', 'is', 'adjusted', 'to', '5.0', '-', '7.0', 'with', 'a', '0.5M-10', 'M', 'sodium', 'hydroxide', 'solution', '.']


In [ ]:
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['<s>', 'ĠPre', 'fer', 'ably', 'Ġ,', 'Ġin', 'Ġstep', 'Ġ2', 'Ġ)', 'Ġof', 'Ġthe', 'Ġmethod', 'Ġof', 'Ġthe', 'Ġpresent', 'Ġin', 'vention', 'Ġ,', 'Ġthe', 'ĠpH', 'Ġof', 'Ġthe', 'Ġdesorption', 'Ġliquid', 'Ġis', 'Ġadjusted', 'Ġto', 'Ġ5', '.', '0', 'Ġ-', 'Ġ7', '.', '0', 'Ġwith', 'Ġa', 'Ġ0', '.', '5', 'M', '-', '10', 'ĠM', 'Ġsodium', 'Ġhydroxide', 'Ġsolution', 'Ġ.', '</s>']


Here the words "Zwingmann" and "sheepmeat" have been split in three subtokens.

This means that we need to do some processing on our labels as the input ids returned by the tokenizer are longer than the lists of labels our dataset contain, first because some special tokens might be added (we can a `[CLS]` and a `[SEP]` above) and then because of those possible splits of words in multiple tokens:

In [ ]:
len(example[f"numeric_tags"]), len(tokenized_input["input_ids"])

(34, 48)

Thankfully, the tokenizer returns outputs that have a `word_ids` method which can help us.

In [ ]:
print(tokenized_input.word_ids())

[None, 0, 0, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 23, 23, 24, 25, 25, 25, 26, 27, 28, 28, 28, 28, 28, 28, 29, 30, 31, 32, 33, None]


As we can see, it returns a list with the same number of elements as our processed input ids, mapping special tokens to `None` and all other tokens to their respective word. This way, we can align the labels with the processed input ids.

In [ ]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example[f"numeric_tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

48 48


Here we set the labels of all special tokens to -100 (the index that is ignored by PyTorch) and the labels of all other tokens to the label of the word they come from. Another strategy is to set the label only on the first token obtained from a given word, and give a label of -100 to the other subtokens from the same word. We propose the two strategies here, just change the value of the following flag:

In [ ]:
label_all_tokens = True

We're now ready to write the function that will preprocess our samples. We feed them to the `tokenizer` with the argument `truncation=True` (to truncate texts that are bigger than the maximum size allowed by the model) and `is_split_into_words=True` (as seen above). Then we align the labels with the token ids using the strategy we picked:

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True, max_length=512)

    labels = []
    for i, label in enumerate(examples[f"numeric_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [ ]:
tokenize_and_align_labels(datasets['train'][:5])

{'input_ids': [[0, 403, 821, 276, 3435, 2973, 280, 31525, 5911, 350, 281, 39734, 464, 20, 8291, 3267, 11897, 31525, 5911, 350, 646, 280, 1731, 1878, 29884, 9148, 10140, 1515, 3173, 10140, 6046, 405, 465, 17, 30093, 3267, 550, 17, 27, 2122, 290, 674, 17, 49957, 1802, 382, 369, 17463, 3180, 600, 5908, 1878, 17686, 3911, 6045, 28297, 3267, 11897, 31525, 5911, 350, 646, 280, 1731, 1878, 29884, 9148, 10140, 1515, 1802, 382, 369, 17463, 3180, 600, 5908, 1878, 17686, 3911, 340, 973, 276, 1531, 369, 36, 17, 2], [0, 531, 1153, 397, 1377, 2278, 354, 8849, 3267, 270, 839, 44171, 660, 345, 10445, 302, 2271, 16, 20, 839, 3227, 660, 2245, 2], [0, 7817, 281, 9271, 5433, 11885, 18351, 3627, 4072, 2], [0, 790, 2245, 10596, 281, 270, 31525, 5911, 350, 281, 39734, 464, 20, 8291, 1787, 302, 7843, 350, 3267, 270, 31525, 5911, 362, 281, 39734, 464, 20, 8291, 1787, 302, 7843, 369, 3267, 270, 31525, 5911, 281, 39734, 464, 21, 8291, 1787, 302, 7843, 465, 3267, 397, 270, 31525, 5911, 281, 39734, 464, 20, 8291, 

To apply this function on all the sentences (or pairs of sentences) in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command.

In [ ]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/162 [00:00<?, ? examples/s]

Map:   0%|          | 0/18 [00:00<?, ? examples/s]

Even better, the results are automatically cached by the 🤗 Datasets library to avoid spending time on this step the next time you run your notebook. The 🤗 Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. 🤗 Datasets warns you when it uses cached files, you can pass `load_from_cache_file=False` in the call to `map` to not use the cached files and force the preprocessing to be applied again.

Note that we passed `batched=True` to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

## Fine-tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it. Since all our tasks are about token classification, we use the `AutoModelForTokenClassification` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us. The only thing we have to specify is the number of labels for our problem (which we can get from the features, as seen before):

In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

# old_model= AutoModelForTokenClassification.from_pretrained(model_checkpoint) 
# model= AutoModelForTokenClassification.from_pretrained("bert-base-uncased", num_labels=len(label_list))
# model.bert=old_model.bert

Some weights of the model checkpoint at /content/RoBERTa-large-PM-M3-Voc/RoBERTa-large-PM-M3-Voc-hf were not used when initializing RobertaForTokenClassification: ['lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at /content/RoBERTa-large-PM-M3-Voc/RoBERTa-large-PM-M3-Voc-hf and are newly initi

The warning is telling us we are throwing away some weights (the `vocab_transform` and `vocab_layer_norm` layers) and randomly initializing some other (the `pre_classifier` and `classifier` layers). This is absolutely normal in this case, because we are removing the head used to pretrain the model on a masked language modeling objective and replacing it with a new head for which we don't have pretrained weights, so the library warns us we should fine-tune this model before using it for inference, which is exactly what we are going to do.

To instantiate a `Trainer`, we will need to define three more things. The most important is the [`TrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [ ]:
datasets['train'].shape

(162, 3)

In [ ]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-{task}-combine-filtered-fintuned",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=10,
    weight_decay=0.01,
    lr_scheduler_type='cosine',
    push_to_hub=True,
)

Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, use the `batch_size` defined at the top of the notebook and customize the number of epochs for training, as well as the weight decay.

The last argument to setup everything so we can push the model to the [Hub](https://huggingface.co/models) regularly during training. Remove it if you didn't follow the installation steps at the top of the notebook. If you want to save your model locally in a name that is different than the name of the repository it will be pushed, or if you want to push your model under an organization and not your name space, use the `hub_model_id` argument to set the repo name (it needs to be the full name, including your namespace: for instance `"sgugger/bert-finetuned-ner"` or `"huggingface/bert-finetuned-ner"`).

Then we will need a data collator that will batch our processed examples together while applying padding to make them all the same size (each pad will be padded to the length of its longest example). There is a data collator for this task in the Transformers library, that not only pads the inputs, but also the labels:

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

The last thing to define for our `Trainer` is how to compute the metrics from the predictions. Here we will load the [`seqeval`](https://github.com/chakki-works/seqeval) metric (which is commonly used to evaluate results on the CONLL dataset) via the Datasets library.

In [ ]:
metric = load_metric("seqeval")

This metric takes list of labels for the predictions and references:

In [ ]:
labels = [label_list[i] for i in example[f"numeric_tags"]]
metric.compute(predictions=[labels], references=[labels])

{'Chemical': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'Lab_value': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

So we will need to do a bit of post-processing on our predictions:
- select the predicted index (with the maximum logit) for each token
- convert it to its string label
- ignore everywhere we set a label of -100

The following function does all this post-processing on the result of `Trainer.evaluate` (which is a namedtuple containing predictions and labels) before applying the metric:

In [ ]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

Note that we drop the precision/recall/f1 computed for each category and only focus on the overall precision/recall/f1/accuracy.

Then we just need to pass all of this along with our datasets to the `Trainer`:

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/content/RoBERTa-large-PM-M3-Voc-hf-finetuned-ner-combine-filtered is already a clone of https://huggingface.co/ktgiahieu/RoBERTa-large-PM-M3-Voc-hf-finetuned-ner-combine-filtered. Make sure you pull the latest changes with `repo.git_pull()`.


We can now finetune our model by just calling the `train` method:

In [ ]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.777274,0.000000,0.000000,0.000000,0.722838
2,No log,0.491469,0.456989,0.361702,0.403800,0.829268
3,No log,0.424630,0.409091,0.344681,0.374134,0.840355
4,No log,0.372463,0.489899,0.412766,0.448037,0.865854
5,No log,0.362829,0.621212,0.523404,0.568129,0.879157
6,No log,0.349211,0.645833,0.527660,0.580796,0.882483
7,No log,0.345299,0.633663,0.544681,0.585812,0.887472
8,No log,0.354643,0.628272,0.510638,0.563380,0.884146
9,No log,0.349130,0.608911,0.523404,0.562929,0.885809
10,No log,0.351061,0.627551,0.523404,0.570766,0.885809


/usr/local/lib/python3.9/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=110, training_loss=0.3939373016357422, metrics={'train_runtime': 76.5287, 'train_samples_per_second': 21.169, 'train_steps_per_second': 1.437, 'total_flos': 961119197196840.0, 'train_loss': 0.3939373016357422, 'epoch': 10.0})

The `evaluate` method allows you to evaluate again on the evaluation dataset or on another dataset:

In [ ]:
trainer.evaluate()

{'eval_loss': 0.3510608375072479,
 'eval_precision': 0.6275510204081632,
 'eval_recall': 0.5234042553191489,
 'eval_f1': 0.5707656612529001,
 'eval_accuracy': 0.885809312638581,
 'eval_runtime': 0.3286,
 'eval_samples_per_second': 54.778,
 'eval_steps_per_second': 6.086,
 'epoch': 10.0}

To get the precision/recall/f1 computed for each category now that we have finished training, we can apply the same function as before on the result of the `predict` method:

In [ ]:
predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

{'Chemical': {'precision': 0.8444444444444444,
  'recall': 0.7450980392156863,
  'f1': 0.7916666666666666,
  'number': 51},
 'Lab_value': {'precision': 0.6666666666666666,
  'recall': 0.6896551724137931,
  'f1': 0.6779661016949153,
  'number': 29},
 'Term': {'precision': 0.5371900826446281,
  'recall': 0.41935483870967744,
  'f1': 0.47101449275362317,
  'number': 155},
 'overall_precision': 0.6275510204081632,
 'overall_recall': 0.5234042553191489,
 'overall_f1': 0.5707656612529001,
 'overall_accuracy': 0.885809312638581}

You can now upload the result of the training to the Hub, just execute this instruction:

In [ ]:
trainer.push_to_hub()

Upload file pytorch_model.bin:   0%|          | 1.00/1.32G [00:00<?, ?B/s]

Upload file runs/Apr23_13-26-02_a567809e38ac/events.out.tfevents.1682256368.a567809e38ac.661.3:   0%|         …

Upload file runs/Apr23_13-24-39_a567809e38ac/events.out.tfevents.1682256295.a567809e38ac.661.0:   0%|         …

Upload file training_args.bin:   0%|          | 1.00/3.56k [00:00<?, ?B/s]

Upload file runs/Apr23_13-26-02_a567809e38ac/1682256368.6715496/events.out.tfevents.1682256368.a567809e38ac.66…

Upload file runs/Apr23_13-24-39_a567809e38ac/events.out.tfevents.1682256324.a567809e38ac.661.2:   0%|         …

Upload file runs/Apr23_13-26-02_a567809e38ac/events.out.tfevents.1682256546.a567809e38ac.661.5:   0%|         …

Upload file runs/Apr23_13-24-39_a567809e38ac/1682256295.5521483/events.out.tfevents.1682256295.a567809e38ac.66…

To https://huggingface.co/ktgiahieu/RoBERTa-large-PM-M3-Voc-hf-finetuned-ner-combine-filtered
   dea011d..b4bd412  main -> main

   dea011d..b4bd412  main -> main

To https://huggingface.co/ktgiahieu/RoBERTa-large-PM-M3-Voc-hf-finetuned-ner-combine-filtered
   b4bd412..ffd2dc9  main -> main

   b4bd412..ffd2dc9  main -> main



'https://huggingface.co/ktgiahieu/RoBERTa-large-PM-M3-Voc-hf-finetuned-ner-combine-filtered/commit/b4bd412c3059010b90bdbc9dff0678df4835216a'

You can now share this model with all your friends, family, favorite pets: they can all load it with the identifier `"your-username/the-name-you-picked"` so for instance:

```python
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained("sgugger/my-awesome-model")
```